In [5]:
# VAE 인코더 네트워크
# 이미지를 잠재 공간상 확률 분포 파라미터로 매핑하는 인코더 네트워크.
# 입력 이미지 x를 z_mean, z_log_va 로 매핑하는 간단한 컨브넷.

import keras
from keras import layers
from keras import backend as K
from keras.models import Model
import numpy as np

img_shape  = (28, 28, 1)
batch_size = 16
latent_dim = 2  # 잠재 공간의 차원: 2D 평면

input_img  = keras.Input(shape = img_shape)

x = layers.Conv2D(32, 3, padding = 'same', activation = 'relu')(input_img)
x = layers.Conv2D(64, 3, padding = 'same', activation = 'relu', strides = (2, 2))(x)
x = layers.Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = layers.Conv2D(64, 3, padding = 'same', activation = 'relu')(x)

shape_before_flattening = K.int_shape(x)

x = layers.Flatten()(x)
x = layers.Dense(32, activation = 'relu')(x)

# Q. 결국 2개의 파라미터로 인코딩 됨.?
#    그냥 Dense layer에 넣으면 자동으로 인코딩이 된다는 것인가.

z_mean    = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

In [8]:
# z_mean, z_log_var을 사용하는 코드.
# 이 두 파라미터가 input_img를 생성한 통계 분포의 파라미터라 가정하고 잠재 공간 포인트 z를 생성.
# 이때 케라스의 벡엔드 기능으로 만든 일련의 코드를 Lambda 층으로 감쌈.
# 케라스는 모든 것이 층이므로, 기본 층을 사용하지 않는 코드는 lambda로 감싸야 함.

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape = (K.shape(z_mean)[0], latent_dim), mean = 0., stddev = 1.)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

W0730 00:21:35.172613  9404 deprecation_wrapper.py:119] From C:\Users\koni1\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.



In [9]:
# 디코더 구현,
# 벡터 z를 이전 특성 맵 차원으로 크기를 바꾸고, 몇 개의 합성곱 층을 사용하여 최종 출력 이미지를 만듬.
# 최종 이미지는 원본 input_img와 차원이 같음.

# int_shape() : 텐서의 크기를 파이썬 튜플로 변환하는 함수
decoder_input = layers.Input(K.int_shape(z)[1:]) 

# 마지막 합성곱 층에서 구한 특성 맵의 크기인 shape_before_flattening은 (None, 14, 14, 64) 임
# 이 크기를 복원해야 하므로, 업샘플링하는 Dense층의 크기는 14 x 14 x 64 = 12,544가 됨. 
x = layers.Dense(np.prod(shape_before_flattening[1:]), activation = 'relu')(decoder_input) # 입력을 업샘플링.

# 인코더 모델의 마지막 flatten 층 직전의 특성 맵과 같은 크기를 가진 특성 맵으로 z의 크기를 바꿈.
x = layers.Reshape(shape_before_flattening[1:])(x)

# Conv2DTranspose 층과 Conv2D 층을 사용하여 z를 원본 입력 이미지와 같은 크기의 특성 맵으로 디코딩함.
# ** Conv2DTranspose
#    입력 값 사이에 0을 추가하여 출력을 업샘플링하는 전치 합성곱(transpose convolution)을 수행.
#    전치 합성곱에서 (14, 14, 64) 크기의 입력이 (28, 28, 32) 크기로 업샘플링 됨. 
#    이따금 전치 합성곱을 역합성곱으로도 부름. 
x = layers.Conv2DTranspose(32, 3, padding    = 'same',
                                  activation = 'relu',
                                  strides = (2, 2))(x)

x         = layers.Conv2D(1, 3, padding = 'same', activation  = 'sigmoid')(x) 
decoder   = Model(decoder_input, x) # decoder_input을 디코딩된 이미지로 변환하는 디코더 모델의 객체를 만듬.
z_decoded = decoder(z)

In [15]:
# 일반적인 샘플 기준의 함수인 loss(y_true, y_pred) 형태는 VAE 이중 손실에 맞지 않음.
# add_loss 내장 메서드를 사용하는 층을 직접 만들어 임의의 손실을 정의
# Layer 클래스의 add_loss() 메서드를 사용하여 추가된 손실은 vae.losses 파이썬 리스트에서 확인할 수 있음.
# k1_loss 계산식의 상수 값이 원래 0.5이지만 여기서는 규제 손실의 양을 조절하기 위해 0.00005가 사용됨.

class CustomVariationalLayer(keras.layers.Layer):
    
    def vae_loss(self, x, z_decoded):
        # x : 원본 `
        # z_decoded : 만들어진 놈. 
        x         = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded) # 재구성 손실 
        kl_loss   = -5e-4 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis = -1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs): # call 메서드가 있는 층을 구성
        x         = inputs[0]
        z_decoded = inputs[1]
        loss      = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x # 이 층을 사용하진 않지만, 무엇가를 반환해야함.

y = CustomVariationalLayer()([input_img, z_decoded])

NameError: name 'z_decoded' is not defined

In [17]:
# VAE 훈련하기
# 층에서 손실을 직접 다루기 때문에 compile 메서드에서 손실을 직접 지정하지 않음.
# 그 결과, 훈련하는 동안 타깃 데이터를 전달하지 않아도 됨.
from keras.datasets import mnist
from keras import Model

# vae = Model(input_img, y) # y는 CustomVariationalLayer() 로 만들어진 층
# vae.compile(optimizer = 'rmsprop', loss = None)
# vae.summary()

(x_train, _), (x_test, y_test) = mnist.load_data()

# 왜 + (1, ) 를 했을까?
x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1, ))
x_test  = x_test.astype('float32')  / 255.
x_test  = x_test.reshape(x_test.shape + (1,))

# vae
vae.fit(x = x_train, y = None, 
        shuffle = True,
        epochs  = 10,
        batch_size = batch_size,
        validation_data = (x_test, None))

NameError: name 'vae' is not defined

In [18]:
import matplotlib.pyplot as plt
from scipy.stats import norm

n = 15
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        z_sample = np.tile(z_sample, batch_size).reshape(batch_size, 2)
        x_decoded = decoder.predict(z_sample, batch_size = batch_size)
        
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
                  j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize = (10, 10))
plt.imshow(figure, cmap = 'Greys_r')
plt.show()

NameError: name 'np' is not defined